## CLASSIFICATION OF MESSAGES AS EITHER SPAM OR NON-SPAM

The purpose of the project is to classify  a dataset od messages as their spam or non-spam, to accurately classify the message the following has to be achieved;
S
To classify messages as spam or non-spam:

1. Learns how humans classify messages.

2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
